In [1]:
import torch
from PIL import Image
import numpy as np
from basicsr.archs.rrdbnet_arch import RRDBNet
from realesrgan import RealESRGANer
import os
import time

In [2]:
input_folder = "inputs"   # folder with input images
output_folder = "outputs" # folder where enhanced images will be saved
os.makedirs(output_folder, exist_ok=True)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device:", torch.cuda.get_device_name(0))

CUDA available: True
Device: NVIDIA GeForce RTX 4050 Laptop GPU


In [4]:
model = RRDBNet(
    num_in_ch=3, num_out_ch=3, num_feat=64,
    num_block=23, num_grow_ch=32, scale=4
)

In [5]:
upsampler = RealESRGANer(
    scale=4,
    model_path='RealESRGAN_x4plus.pth',
    model=model,
    tile=0,           # adjust for memory if needed
    tile_pad=10,
    pre_pad=0,
    half=True,        # use FP16 for faster GPU inference
    device=device
)

c:\Users\91630\AppData\Local\Programs\Python\Python310\lib\site-packages\realesrgan\utils.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loadnet = torch.load(model_pat

In [6]:
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)

        print(f"Processing: {filename}")
        img = Image.open(input_path).convert('RGB')
        img_np = np.array(img)

        start_time = time.time()
        output, _ = upsampler.enhance(img_np, outscale=4)
        end_time = time.time()

        Image.fromarray(output).save(output_path)
        print(f"✅ Saved: {output_path}")
        print(f"⏱ Time taken: {end_time - start_time:.2f} seconds\n")

print("🎉 All images processed successfully!")

Processing: Screenshot 2025-10-24 101736.png
✅ Saved: outputs\Screenshot 2025-10-24 101736.png
⏱ Time taken: 0.68 seconds

🎉 All images processed successfully!
